Przygotowanie środowiska

In [4]:
import pandas as pd
import matplotlib as plt

In [5]:
fields = ['P_ROKDYP',"P_N","P_POZIOM", # wartosci filtrujace
"P_E_ZAR_STUD_P5","P_E_ZAR_NSTUD_P5","P_E_ZAR_P5", #potencjalne dane o zarobkach absolwentow po 5 latach
"P_E_ZAR_DOSW_P5","P_E_ZAR_NDOSW_P5",
"P_ME_ZAR_P5","P_ME_ZAR_DOSW_P5","P_ME_ZAR_NDOSW_P5",
"P_ME_ZAR_STUD_P5","P_ME_ZAR_NSTUD_P5",
"P_E_ZAR_ETAT_P5","P_E_ZAR_ETAT_DOSW_P5","P_E_ZAR_ETAT_NDOSW_P5",
"P_E_ZAR_ETAT_STUD_P5","P_E_ZAR_ETAT_NSTUD_P5",
"P_ME_ZAR_ETAT_P5","P_ME_ZAR_ETAT_DOSW_P5","P_ME_ZAR_ETAT_NDOSW_P5",
"P_ME_ZAR_ETAT_STUD_P5","P_ME_ZAR_ETAT_NSTUD_P5",
'P_NAZWA_UCZELNI',"P_NAZWA_KIERUNKU_PELNA" #dane o uczelni i kierunku
,"P_KIERUNEK_ID","P_FORMA","P_DZIEDZINA","P_WOJ"] 
data = pd.read_csv('../dane/graduates-major-data.csv',sep=";",usecols=fields,decimal=',')

Wstępna filtracja, kierunki z "informatyka" w nazwie

In [6]:
data = data[data['P_NAZWA_KIERUNKU_PELNA'].str.contains('informatyka',case=False,regex=False)]

Zgrupowanie danych
stopien studiow -> uczelnia -> kierunek -> forma

In [7]:
# stopien studiow -> uczelnia -> kierunek -> forma
# P_WOJ dla informacji pozniej
data_wages_degree = data.groupby(
    ["P_POZIOM","P_NAZWA_UCZELNI","P_KIERUNEK_ID","P_NAZWA_KIERUNKU_PELNA","P_FORMA","P_DZIEDZINA","P_WOJ"]
    ).agg({
    "P_N" :'sum',"P_ME_ZAR_P5":'mean'
}).reset_index()
data_wages_degree = data_wages_degree[data_wages_degree["P_N"]>=25]

In [8]:
wojewodztwa = {
    14:"Mazowieckie", 12:"Małopolskie", 24:"Śląskie",30:"Wielkopolskie",
    2:"Dolnoślaśkie",22:"Pomorskie",6:"Lubelskie",10:"Łódzkie",
    4:"Kujawsko-Pomorskie",32:"Zachodniopomorskie",18:"Podkarpackie",
    20:"Podlaskie",28:"Warmińsko-mazurskie",16:"Opolskie",
    26:"Świętokrzyskie", 8:"Lubuskie"
}
data_wages_degree['P_WOJ']=data_wages_degree['P_WOJ'].map(wojewodztwa)
data_wages_degree = data_wages_degree.reset_index().drop(columns='index')

1/3 z kierunków nie posiada inforamcji o zarobkach

In [9]:
data_wages_degree.isna().sum()

P_POZIOM                    0
P_NAZWA_UCZELNI             0
P_KIERUNEK_ID               0
P_NAZWA_KIERUNKU_PELNA      0
P_FORMA                     0
P_DZIEDZINA                 0
P_WOJ                       0
P_N                         0
P_ME_ZAR_P5               214
dtype: int64

Uzupełnienie brakujących wartości będzie polegało na wstawieniu średnich zarobków dla informatyków z danego województwa w zależności od stopnia ukonczonych studiów

In [10]:
# srednie zarobki w wojewodztwach w zaleznosci od stopnia studiow
data_wages_woj = data_wages_degree.groupby(["P_WOJ","P_POZIOM"]
    ).agg({"P_ME_ZAR_P5":'mean'}).reset_index()
data_wages_woj.head() # 32 wartosci, dla kuj-pom magisterskich jedyny brak danych

,P_WOJ,P_POZIOM,P_ME_ZAR_P5
0,Dolnoślaśkie,1,7347.531304
1,Dolnoślaśkie,2,9423.585769
2,Kujawsko-Pomorskie,1,5605.722500
3,Kujawsko-Pomorskie,2,NaN
4,Lubelskie,1,5110.387917


uzupelnienie danych o zarobkach

In [11]:
import numpy as np
# awaryjnie kopia by nie nadpisywac tych nie zmienionych danych
data_wages_degree_filled = data_wages_degree.copy()
# indexy pustych wierszy
missing = data_wages_degree_filled['P_ME_ZAR_P5'].isna()

# zmapowanie wojewodztw i stopnia jak w data_wages_woj
condWoj = [
    "Dolnoślaśkie","Kujawsko-Pomorskie","Lubelskie","Lubuskie",
    "Mazowieckie","Małopolskie","Opolskie","Podkarpackie",
    "Podlaskie","Pomorskie","Warmińsko-mazurskie",
    "Wielkopolskie","Zachodniopomorskie","Łódzkie",
    "Śląskie","Świętokrzyskie"
]
condDegr = ["1","2"]

#przygotowanie listy warunkow jezeli rzad jest pusty & woj & stopien 32 przypadki
condList = [np.logical_and(data_wages_degree_filled.loc[missing,'P_ME_ZAR_P5'],
            np.logical_and(data_wages_degree_filled.loc[missing,'P_POZIOM'] == degr,
            data_wages_degree_filled.loc[missing,'P_WOJ'] == woj))       
             for woj in condWoj for degr in condDegr  ]
# lista przypadkow
choicelist = data_wages_woj['P_ME_ZAR_P5'].to_list()

# zmapowanie tych przypadkow na zbiorze 
data_wages_degree_filled.loc[missing,"P_ME_ZAR_P5"] = np.select(condList, choicelist)

Pozostał w ten sposób 1 pusty przypadek

In [12]:
# data_wages_degree_filled[data_wages_degree_filled['P_WOJ']=="Kujawsko-Pomorskie"].tail()
data_wages_degree_filled.isna().sum()

P_POZIOM                  0
P_NAZWA_UCZELNI           0
P_KIERUNEK_ID             0
P_NAZWA_KIERUNKU_PELNA    0
P_FORMA                   0
P_DZIEDZINA               0
P_WOJ                     0
P_N                       0
P_ME_ZAR_P5               1
dtype: int64

# POTEM

TODO

JAK SIE KSZTALTUJA ZAROBKI wszystkich absolwentow


WSKAZNIK - taki stosunek zrobic
ZAROBKIKIERUNKU/SREDNIEZAROBKIMAGISTROW